In [360]:
from pathlib import Path
import re

import pandas as pd

DATA_PATH = "data"
RE_FILE_YEAR = re.compile(r"resultat-ansokningsomgang-(20\d{2})\.xlsx$")

current_dir = globals()["_dh"][0]  # CWD for jupyter
path_data = Path(current_dir) / DATA_PATH

In [361]:
# Tabell 1 / `sheet_name=2` / Beviljade ansökningar

# EXTRACT

df_2020 = pd.read_excel(path_data / "resultat-ansokningsomgang-2020.xlsx", sheet_name=2)
df_2020["Ansökningsomgång"] = 2020
df_2020 = df_2020.rename(columns={"Flera studiekommuner": "Flera kommuner"})
df_2020 = df_2020.drop(columns=["Antal län"])

df_2021 = pd.read_excel(path_data / "resultat-ansokningsomgang-2021.xlsx", sheet_name=2)
df_2021["Ansökningsomgång"] = 2021
df_2021 = df_2021.rename(columns={"Flera studiekommuner": "Flera kommuner"})
df_2021 = df_2021.drop(columns=["Antal län"])

df_2022 = pd.read_excel(path_data / "resultat-ansokningsomgang-2022.xlsx", sheet_name=2)
df_2022["Ansökningsomgång"] = 2022

df_2023 = pd.read_excel(path_data / "resultat-ansokningsomgang-2023.xlsx", sheet_name=2, skiprows=5)
df_2023["Ansökningsomgång"] = 2023

df_2024 = pd.read_excel(path_data / "resultat-ansokningsomgang-2024.xlsx", sheet_name=2, skiprows=5)
df_2024["Ansökningsomgång"] = 2024

annual_datasets = [df_2020, df_2021, df_2022, df_2023, df_2024]

df_tabell1 = pd.concat(annual_datasets, ignore_index=True)


# TRANSFORM

df_tabell1["Flera kommuner"] = df_tabell1["Flera kommuner"].map({"Ja": True, "Nej": False})
if df_tabell1["Flera kommuner"].isna().sum() > 0:
    raise ValueError(f"Found {df_tabell1['Flera kommuner'].isna().sum()} NaN values in 'Flera kommuner'")


# LOAD

year_min = df_tabell1["Ansökningsomgång"].min()
year_max = df_tabell1["Ansökningsomgång"].max()

df_tabell1.to_csv(path_data / f"resultat-ansokningsomgang-{year_min}-{year_max}-beviljade.csv")

# RESULT

df_tabell1


,Utbildningsområde,Utbildningsnamn,Län,Kommun,Flera kommuner,Antal kommuner,YH-poäng,Utbildningsanordnare administrativ enhet,Diarienummer,Ansökningsomgång
0,Data/IT,.NET Utvecklare,Västra Götaland,Göteborg,False,1,400,Plushögskolan AB - Teknikhögskolan,MYH 2020/4403,2020
1,Data/IT,.NET-utvecklare,Stockholm,Stockholm,False,1,400,IT-Högskolan Stockholm AB,MYH 2020/5766,2020
2,Data/IT,.NET-utvecklare,Västra Götaland,Göteborg,False,1,400,IT-Högskolan Sverige AB,MYH 2020/5764,2020
3,Data/IT,Agil Javautvecklare,Gävleborg,Gävle,False,1,400,Edugrade AB,MYH 2020/5827,2020
4,Data/IT,Agil projektledare,Skåne,Malmö,False,1,225,IHM Business School AB Malmö,MYH 2020/5354,2020
...,...,...,...,...,...,...,...,...,...,...
2146,Transporttjänster,Lokförare,Västra Götaland,Göteborg,False,1,220,TCC Sverige AB,MYH 2024/4150,2024
2147,Transporttjänster,Lokförare Malmö,Skåne,Malmö,False,1,220,ProTrain Utbildning AB,MYH 2024/3991,2024
2148,Transporttjänster,Lokförare Stockholm,Stockholm,Solna,False,1,220,ProTrain Utbildning AB,MYH 2024/3996,2024
2149,Övrigt,Hantverkslärling - smala och traditionella han...,Dalarna,Leksand,False,1,500,Stiftelsen Hantverk & Utbildning,MYH 2024/3586,2024


In [362]:
del df_2020, df_2021, df_2022, df_2023, df_2024, annual_datasets

In [363]:
# EXTRACT

df_2020 = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2020.xlsx", sheet_name=5, usecols=["Diarienummer", "Kommun"]
)
df_2021 = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2021.xlsx", sheet_name=5, usecols=["Diarienummer", "Kommun"]
)
df_2022 = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2022.xlsx", sheet_name=5, usecols=["Diarienummer", "Kommun"]
)

df_2023a = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2023.xlsx", sheet_name=5, skiprows=5, usecols=["Diarienummer", "Kommun"]
)
df_2023b = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2023.xlsx",
    sheet_name=4,
    skiprows=5,
    usecols=["Diarienummer", "Kommun", "Antal kommuner"],
)
df_2023b = df_2023b[df_2023b["Antal kommuner"] <= 1].drop(columns=["Antal kommuner"])

df_2024a = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2024.xlsx", sheet_name=5, skiprows=5, usecols=["Diarienummer", "Kommun"]
)
df_2024b = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2024.xlsx",
    sheet_name=4,
    skiprows=5,
    usecols=["Diarienummer", "Kommun", "Antal kommuner"],
)
df_2024b = df_2024b[df_2024b["Antal kommuner"] <= 1].drop(columns=["Antal kommuner"])

annual_datasets = [df_2020, df_2021, df_2022, df_2023a, df_2023b, df_2024a, df_2024b]
df_diarie_kommun = pd.concat(annual_datasets, ignore_index=True)


# TRANSFORM

df_diarie_kommun = df_diarie_kommun.drop_duplicates().sort_values(by="Diarienummer").reset_index(drop=True)


# LOAD

df_diarie_kommun.to_csv(path_data / "resultat-ansokningsomgang-2020-2024-diarie_kommun.csv")


# RESULT

df_diarie_kommun


,Kommun,Diarienummer
0,Gävle,MYH 2020/1698
1,Sundsvall,MYH 2020/1698
2,Umeå,MYH 2020/1698
3,Östersund,MYH 2020/1698
4,Nyköping,MYH 2020/3237
...,...,...
8584,Stockholm,MYH 2024/4273
8585,Umeå,MYH 2024/4274
8586,Stockholm,MYH 2024/4275
8587,Göteborg,MYH 2024/4276


In [364]:
del df_2020, df_2021, df_2022, df_2023a, df_2023b, df_2024a, df_2024b, annual_datasets

In [365]:
df_antal_kommuner = df_diarie_kommun.groupby("Diarienummer")["Kommun"].count().reset_index(name="Antal kommuner")

df_antal_kommuner

,Diarienummer,Antal kommuner
0,MYH 2020/1698,4
1,MYH 2020/3237,1
2,MYH 2020/3238,1
3,MYH 2020/3330,1
4,MYH 2020/3475,1
...,...,...
6452,MYH 2024/4273,1
6453,MYH 2024/4274,1
6454,MYH 2024/4275,1
6455,MYH 2024/4276,1


In [366]:
# EXTRACT
drop_cols = [f"Beviljade platser utbildningsomgång {n}" for n in range(1, 6)]


df_2020a = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2020.xlsx",
    sheet_name=4,
    usecols=["Diarienummer", "Sökta utbildningsomgångar", "Beviljade utbildningsomgångar"],
)
df_2020 = pd.read_excel(path_data / "resultat-ansokningsomgang-2020.xlsx", sheet_name=5)
df_2020 = df_2020.drop(columns=["Beviljade platser per utbildningsomgång"])
df_2020 = df_2020.rename(columns={"Flera studiekommuner": "Flera kommuner", "Antal studiekommuner": "Antal kommuner"})

df_2020 = df_2020.merge(df_2020a, on="Diarienummer", how="left")
df_2020["Ansökningsomgång"] = 2020


df_2021a = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2021.xlsx",
    sheet_name=4,
    usecols=["Diarienummer", "Sökta utbildningsomgångar", "Beviljade utbildningsomgångar"],
)
df_2021 = pd.read_excel(path_data / "resultat-ansokningsomgang-2021.xlsx", sheet_name=5)
df_2021 = df_2021.drop(columns=drop_cols)
df_2021 = df_2021.rename(columns={"Flera studiekommuner": "Flera kommuner", "Antal studiekommuner": "Antal kommuner"})

df_2021 = df_2021.merge(df_2021a, on="Diarienummer", how="left")
df_2021["Ansökningsomgång"] = 2021


df_2022a = pd.read_excel(
    path_data / "resultat-ansokningsomgang-2022.xlsx",
    sheet_name=4,
    usecols=["Diarienummer", "Sökta utbildningsomgångar", "Beviljade utbildningsomgångar"],
)
df_2022 = pd.read_excel(path_data / "resultat-ansokningsomgang-2022.xlsx", sheet_name=5)
df_2022 = df_2022.drop(columns=drop_cols)

df_2022 = df_2022.merge(df_2022a, on="Diarienummer", how="left")
df_2022["Ansökningsomgång"] = 2022


df_2023 = pd.read_excel(path_data / "resultat-ansokningsomgang-2023.xlsx", sheet_name=4, skiprows=5)
df_2023["Ansökningsomgång"] = 2023
df_2023 = df_2023.drop(columns=drop_cols)


df_2024 = pd.read_excel(path_data / "resultat-ansokningsomgang-2024.xlsx", sheet_name=4, skiprows=5)
df_2024["Ansökningsomgång"] = 2024
df_2024 = df_2024.drop(columns=drop_cols)


annual_datasets = [df_2020, df_2021, df_2022, df_2023, df_2024]

df_beslut = pd.concat(annual_datasets, ignore_index=True)


# TRANSFORM

df_beslut['Beviljade platser totalt'] = df_beslut['Beviljade platser totalt'].astype(int)


# Drop columns with incomplete data
drop_cols_nan = [
    "Antal län",
    # "Antal kommuner",  # replaced with agg on df_diarie_kommun
    "Typ av examen",
    "SUN5 inriktning",
    "SUN5 inriktning namn",
    "SeQF nivå",
    "Smalt yrkesområde",
]
df_beslut = df_beslut.drop(columns=drop_cols_nan)


# Merge and validate 'Antal kommuner'
df_beslut = df_beslut.merge(df_antal_kommuner, on="Diarienummer", how="left")

if not df_beslut["Antal kommuner_x"].equals(df_beslut["Antal kommuner_y"]):
    raise ValueError(
        f"Column mismatch: {(df_beslut['Antal kommuner_x'] != df_beslut['Antal kommuner_y']).sum()} differences found"
    )

df_beslut["Antal kommuner"] = df_beslut["Antal kommuner_x"]
df_beslut = df_beslut.drop(["Antal kommuner_x", "Antal kommuner_y"], axis=1)


# Convert 'Flera kommuner' to bool
df_beslut["Flera kommuner"] = df_beslut["Flera kommuner"].map({"Ja": True, "Nej": False})
if df_beslut["Flera kommuner"].isna().sum() > 0:
    raise ValueError(f"Found {df_beslut['Flera kommuner'].isna().sum()} NaN values in 'Flera kommuner'")

# Standardization of 'Beslut' and convert to bool
df_beslut["Beslut"] = df_beslut["Beslut"].map({"Beviljad": True, "Ej beviljad": False, "Avslag": False})
if df_beslut["Beslut"].isna().sum() > 0:
    raise ValueError(f"Found {df_beslut['Beslut'].isna().sum()} NaN values in 'Beslut'")


# LOAD

df_beslut.to_csv(path_data / "resultat-ansokningsomgang-2020-2024-beslut.csv")


# RESULT

df_beslut

,Utbildningsområde,Utbildningsnamn,Län,Kommun,Flera kommuner,YH-poäng,Studieform,Studietakt %,Utbildningsanordnare administrativ enhet,Huvudmannatyp,Sökta platser per utbildningsomgång,Sökta platser totalt,Beviljade platser totalt,Diarienummer,Beslut,Sökta utbildningsomgångar,Beviljade utbildningsomgångar,Ansökningsomgång,Antal kommuner
0,Data/IT,.NET Developer,Jämtland,Östersund,True,425,Bunden,100,KYH AB,Privat,7,35,0,MYH 2020/4419,False,5,0,2020,5
1,Data/IT,.NET Developer,Kronoberg,Växjö,True,425,Bunden,100,KYH AB,Privat,8,40,0,MYH 2020/4419,False,5,0,2020,5
2,Data/IT,.NET Developer,Skåne,Malmö,False,430,Bunden,100,KYH AB Malmö,Privat,35,105,0,MYH 2020/4482,False,3,0,2020,1
3,Data/IT,.NET Developer,Stockholm,Norrtälje,True,425,Bunden,100,KYH AB,Privat,9,45,0,MYH 2020/4419,False,5,0,2020,5
4,Data/IT,.NET Developer,Södermanland,Katrineholm,True,425,Bunden,100,KYH AB,Privat,7,35,0,MYH 2020/4419,False,5,0,2020,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7635,Transporttjänster,Integrerad Trafikflygarutbildning Statsstöd,Västra Götaland,Göteborg,False,500,Bunden,100,Svensk Pilotutbildning AB,Privat,20,100,0,MYH 2024/3675,False,5,0,2024,1
7636,Transporttjänster,Trafikflygare med inriktning hållbarhet och pi...,Västerbotten,Skellefteå,False,500,Bunden,100,Green Flight Academy AB,Privat,24,120,0,MYH 2024/3415,False,5,0,2024,1
7637,Transporttjänster,UAS-operatör inom Samhällsbyggnad,Flera kommuner,Flera kommuner,True,230,Distans,100,Folkuniversitetet - Stiftelsen Kursverksamhete...,Privat,30,90,0,MYH 2024/3326,False,3,0,2024,5
7638,Övrigt,Hantverkslärling - smala och traditionella han...,Dalarna,Leksand,False,500,Distans,100,Stiftelsen Hantverk & Utbildning,Privat,35,175,175,MYH 2024/3586,True,5,5,2024,1


In [367]:
del df_2020, df_2021, df_2022, df_2023, df_2024, annual_datasets